In [1]:
from datasets import load_dataset

dataset = load_dataset("bigcode/the-stack-dedup",split = 'train', data_dir="data/python")

Resolving data files:   0%|          | 0/144 [00:00<?, ?it/s]

In [2]:
acceptable = [
    'pandas',
    'sklearn',
    'matplotlib',
    'plotly'
]

In [3]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

In [6]:
with open('data-analytics.jsonl', 'w') as fopen:
    for sample in tqdm(iter(dataset)):
        if any([a in sample["content"] for a in acceptable]):
            fopen.write(f'{json.dumps(sample)}\n')

12962249it [1:03:00, 3429.10it/s]


In [7]:
!wc -l data-analytics.jsonl

1228880 data-analytics.jsonl


In [8]:
split_by = 100000

!mkdir partitions-data-analytics

In [9]:
index = 0
count = 0
a = open(f'partitions-data-analytics/{index}.jsonl', 'w')

with open('data-analytics.jsonl') as fopen:
    for l in tqdm(fopen):
        a.write(l)
        a.flush()
        count += 1
        if count >= split_by:
            a.close()
            index += 1
            count = 0
            a = open(f'partitions-data-analytics/{index}.jsonl', 'w')
            
a.close()

1228880it [00:42, 28707.93it/s]


In [10]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('mistralai/Mixtral-8x7B-Instruct-v0.1')

In [15]:
def loop(files):
    files, index = files
    tokenizer = AutoTokenizer.from_pretrained('mistralai/Mixtral-8x7B-Instruct-v0.1')
    with open(f'filtered-data-analytics-{index}.jsonl', 'w') as fopen_l:
        for f in files:
            with open(f) as fopen:
                for l in tqdm(fopen):
                    l = json.loads(l)
                    if len(tokenizer(l['content'])['input_ids']) > 5000:
                        continue

                    fopen_l.write(f'{json.dumps(l)}\n')

In [16]:
from glob import glob

files = sorted(glob('partitions-data-analytics/*.jsonl'), key = lambda x: int(x.split('/')[-1].replace('.jsonl', '')))
files

['partitions-data-analytics/0.jsonl',
 'partitions-data-analytics/1.jsonl',
 'partitions-data-analytics/2.jsonl',
 'partitions-data-analytics/3.jsonl',
 'partitions-data-analytics/4.jsonl',
 'partitions-data-analytics/5.jsonl',
 'partitions-data-analytics/6.jsonl',
 'partitions-data-analytics/7.jsonl',
 'partitions-data-analytics/8.jsonl',
 'partitions-data-analytics/9.jsonl',
 'partitions-data-analytics/10.jsonl',
 'partitions-data-analytics/11.jsonl',
 'partitions-data-analytics/12.jsonl']

In [17]:
import mp
mp.multiprocessing(files, loop, cores = min(len(files), 12), returned = False)

100000it [24:19, 68.50it/s]
100000it [24:51, 67.05it/s]
100000it [25:58, 64.15it/s] 
100000it [26:45, 62.30it/s]
100000it [26:56, 61.87it/s]
100000it [26:59, 61.76it/s]
100000it [27:00, 61.71it/s]
100000it [27:04, 61.54it/s]
100000it [27:05, 61.51it/s]
100000it [27:24, 60.79it/s]
100000it [27:34, 60.42it/s]
100000it [29:04, 57.32it/s]
28880it [06:25, 74.86it/s] 


In [20]:
!ls -lh filtered-data-analytics-*.jsonl

-rw-r--r-- 1 ubuntu ubuntu 492M Dec 31 07:34 filtered-data-analytics-0.jsonl
-rw-r--r-- 1 ubuntu ubuntu 493M Dec 31 07:34 filtered-data-analytics-1.jsonl
-rw-r--r-- 1 ubuntu ubuntu 494M Dec 31 07:34 filtered-data-analytics-10.jsonl
-rw-r--r-- 1 ubuntu ubuntu 493M Dec 31 07:33 filtered-data-analytics-11.jsonl
-rw-r--r-- 1 ubuntu ubuntu 142M Dec 31 07:37 filtered-data-analytics-12.jsonl
-rw-r--r-- 1 ubuntu ubuntu 494M Dec 31 07:31 filtered-data-analytics-2.jsonl
-rw-r--r-- 1 ubuntu ubuntu 494M Dec 31 07:34 filtered-data-analytics-3.jsonl
-rw-r--r-- 1 ubuntu ubuntu 494M Dec 31 07:33 filtered-data-analytics-4.jsonl
-rw-r--r-- 1 ubuntu ubuntu 493M Dec 31 07:34 filtered-data-analytics-5.jsonl
-rw-r--r-- 1 ubuntu ubuntu 495M Dec 31 07:31 filtered-data-analytics-6.jsonl
-rw-r--r-- 1 ubuntu ubuntu 495M Dec 31 07:36 filtered-data-analytics-7.jsonl
-rw-r--r-- 1 ubuntu ubuntu 491M Dec 31 07:34 filtered-data-analytics-8.jsonl
-rw-r--r-- 1 ubuntu ubuntu 495M Dec 31 07:33 filtered-data-analytics-9.js

In [23]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
for f in glob('filtered-data-analytics-*.jsonl'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/mixtral-python-data-analytics-instructions',
        repo_type='dataset',
    )

filtered-data-analytics-0.jsonl


filtered-data-analytics-0.jsonl:   0%|          | 0.00/516M [00:00<?, ?B/s]